### Experiment Tracking with MLflow

MLflow is a powerful tool for managing and tracking machine learning experiments, helping to streamline workflows and analyze results efficiently. In this notebook, we will explore how to use MLflow to log and evaluate models, store essential data, and analyze performance across multiple experiments.

## 1 tracking 

In [2]:
import mlflow
import mlflow.sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import os

In [3]:
# Start an MLflow run
with mlflow.start_run():
    # Log hyperparameters
    mlflow.log_param("learning_rate", 0.01)
    mlflow.log_param("epochs", 10)

    # Log metrics (in a loop simulating training)
    for epoch in range(10):
        accuracy = 0.8 + epoch * 0.02  # Simulated accuracy improvement
        mlflow.log_metric("accuracy", accuracy, step=epoch)

    # Log an artifact (a result file, model, etc.)
    with open("output.txt", "w") as f:
        f.write("This is a test artifact.")
    mlflow.log_artifact("output.txt")

## 2 tracking

In [4]:
# Import necessary libraries
import mlflow
import mlflow.sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import os

In [5]:
# Load the Iris dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [6]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Set hyperparameters for Logistic Regression
learning_rate = 0.01  # In LogisticRegression, this can be interpreted as the inverse of regularization strength
epochs = 10
penalty = 'l2'

In [8]:
# Start an MLflow run
with mlflow.start_run():
    # Log hyperparameters
    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("epochs", epochs)
    mlflow.log_param("penalty", penalty)

    # Initialize and train the model
    model = LogisticRegression(penalty=penalty, solver='lbfgs', max_iter=1, warm_start=True)

    # Train over multiple "epochs" (re-fitting the model)
    for epoch in range(epochs):
        model.fit(X_train, y_train)

        # Predict and calculate accuracy
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        # Log the accuracy as a metric
        mlflow.log_metric("accuracy", accuracy, step=epoch)
        print(f"Epoch {epoch+1}/{epochs} - Accuracy: {accuracy:.4f}")

    # Save the model as an artifact
    model_filename = "logistic_regression_model.pkl"
    mlflow.sklearn.save_model(model, model_filename)

    # Log the saved model as an artifact
    mlflow.log_artifact(model_filename)

    # Save and log an additional artifact (like a text file)
    with open("output.txt", "w") as f:
        f.write("Logistic Regression model for Iris dataset.")
    mlflow.log_artifact("output.txt")

    print("Run complete. Check MLflow UI for details.")

# Instructions to view the results in MLflow UI:
# After running the script, type "mlflow ui" in your terminal to start the UI, and navigate to http://localhost:5000


c:\Users\kkhal\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\kkhal\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Epoch 1/10 - Accuracy: 0.3667
Epoch 2/10 - Accuracy: 0.6333
Epoch 3/10 - Accuracy: 0.7000
Epoch 4/10 - Accuracy: 0.6333
Epoch 5/10 - Accuracy: 0.7000
Epoch 6/10 - Accuracy: 0.6333
Epoch 7/10 - Accuracy: 0.7000
Epoch 8/10 - Accuracy: 0.6667
Epoch 9/10 - Accuracy: 0.7000
Epoch 10/10 - Accuracy: 0.8333
Run complete. Check MLflow UI for details.


## full project with autologging


In [9]:
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris


mlflow.set_experiment("Autolog Iris_Classification_Experiment")

# Enable autologging
mlflow.sklearn.autolog()

# Load dataset
data = load_iris()
X = data.data
y = data.target

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a model
with mlflow.start_run():
    model = LogisticRegression(max_iter=200)
    model.fit(X_train, y_train)
    
    # No need for manual logging; it's handled by autologging


In [10]:
import mlflow
logged_model = 'runs:/9f9139606a3f44538936aeb55822a097/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(X_test))

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])

In [11]:
import mlflow
logged_model = 'runs:/48c5723dc52c4158a3ab0793e7c7193d/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(X_test)

MlflowException: Run '48c5723dc52c4158a3ab0793e7c7193d' not found

In [12]:
import mlflow
import mlflow.keras
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist


mlflow.set_experiment("Autolog_Mnist_Experiment")

# Enable autologging for Keras
mlflow.keras.autolog()

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize pixel values

# Define a simple neural network model
model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model within an MLflow run
with mlflow.start_run():
    model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

# After training, you can access the logged data in the MLflow UI


2025/04/03 18:00:46 WARNING mlflow.utils.autologging_utils: MLflow keras autologging is known to be compatible with 3.0.2 <= keras <= 3.8.0, but the installed version is 3.9.1. If you encounter errors during autologging, try upgrading / downgrading keras to a compatible version, or try upgrading MLflow.
c:\Users\kkhal\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8782 - loss: 0.4344 - val_accuracy: 0.9600 - val_loss: 0.1336
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9671 - loss: 0.1142 - val_accuracy: 0.9695 - val_loss: 0.1035
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9773 - loss: 0.0750 - val_accuracy: 0.9725 - val_loss: 0.0898
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9835 - loss: 0.0559 - val_accuracy: 0.9779 - val_loss: 0.0766
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9880 - loss: 0.0399 - val_accuracy: 0.9798 - val_loss: 0.0704


## GridSearchCV 

In [13]:
import mlflow
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Set experiment name in MLflow
experiment_name = "GridSearch_RF_Experiment"
mlflow.set_experiment(experiment_name)

# Load the dataset
data = load_iris()
X = data.data
y = data.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Set up the Random Forest model
rf = RandomForestClassifier()

# Define the parameters for the GridSearch experiment
param_grid = {
    "n_estimators": [10, 50, 100],
    "max_depth": [3, 5, 10],
    "min_samples_split": [2, 5, 10]
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3)

# Start an MLflow run within a specific experiment
with mlflow.start_run():
    # Run GridSearch
    grid_search.fit(X_train, y_train)

    # Log parameters for each individual trial
    for i, params in enumerate(grid_search.cv_results_["params"]):
        accuracy = grid_search.cv_results_["mean_test_score"][i]
        
        # Log each trial with its trial number
        for key, value in params.items():
            mlflow.log_param(f"trial_{i}_{key}", value)
        
        # Log the accuracy of each trial as a metric
        mlflow.log_metric(f"trial_{i}_accuracy", accuracy)

    # Log the best model with the best parameters
    best_model = grid_search.best_estimator_
    mlflow.sklearn.log_model(best_model, "best_rf_model")

    # Print the results
    print("Best parameters found: ", grid_search.best_params_)
    print("Best accuracy score: ", grid_search.best_score_)

    # Results can be viewed in the MLflow UI after running the code at the link: http://localhost:5000


2025/04/03 18:03:10 INFO mlflow.sklearn.utils: Logging the 5 best runs, 22 runs will be omitted.
2025/04/03 18:03:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Best parameters found:  {'max_depth': 3, 'min_samples_split': 10, 'n_estimators': 10}
Best accuracy score:  0.9619047619047619
